In [2]:
pip install rapidfuzz

Note: you may need to restart the kernel to use updated packages.


In [1]:
#Purpose: To identify list of organisations which are UK universities
#List of EU universities sourced from https://www.eter-project.com/#/search

#module for working with datasets
import pandas as pd

#module for reshaping table
import numpy as np

#module for natural language processing
import nltk

#for dealing with stopwords e.g. and, or, in, at, is
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('english')

from rapidfuzz import process, utils as fuzz_utils

#making data frame
Universities = pd.read_excel('20200912 University List.xlsx')
Projects = pd.read_excel('20200904 UOA23 Worktribe Projects.xlsx')

Universities.head()
#Projects

[nltk_data] Downloading package stopwords to C:\Users\Si
[nltk_data]     Ning\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Institution Name,English Institution Name,Country Code
0,A Tan Kapuja Buddhista Főiskola (TKBF),"""The Gate of the Teaching"" Buddhist College",HU
1,AA School of Architecture,AA School of Architecture,UK
2,Aalborg Universitet,Aalborg University,DK
3,Aalto-yliopisto,Aalto University,FI
4,Aarhus Universitet,Aarhus University,DK


In [2]:
Universities.drop(['English Institution Name'], axis=1, inplace=True)

#renaming the column headings
Universities.columns = Universities.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

#make all text in column lowercase
Universities["institution_name"] = Universities["institution_name"].str.lower()

#the w means any letter, the s means any whitespace, the d+ means numbers and the ^ means 'not' so code below means
#remove all not letters, numbers and spaces from the column
Universities["institution_name"] = Universities["institution_name"].str.replace(r'[^a-z\s\d+]', "")

#remove words if less than 2 letters long
Universities['institution_name'] = Universities['institution_name'].apply(lambda x: ' '.join([word for word in x.split() if len(word)>1]))

#remove stopwords
Universities['institution_name'] = Universities['institution_name'].apply(lambda x: ' '.join([word for word in x.split() if not word in stopwords.words('english')]))

Universities

,institution_name,country_code
0,tan kapuja buddhista fiskola tkbf,HU
1,aa school architecture,UK
2,aalborg universitet,DK
3,aaltoyliopisto,FI
4,aarhus universitet,DK
...,...,...
3643,,BG
3644,,BG
3645,,BG
3646,,BG


In [3]:
Projects.drop(['WT_Record_Status','UOA23','Non-UOA dept collabs','External collabs','Covid'], axis=1, inplace=True)

#renaming the column headings
Projects.columns = Projects.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

#break text to columns
Separate_orgs = Projects ["other_organisations"].str.split("; ", n = -1, expand = True)

Projects = pd.concat([Projects, Separate_orgs], axis=1)

Projects

,project_id,other_organisations,0,1,2,3,4,5,6,7,8
0,560204,3IE,3IE,None,None,None,None,None,None,None,None
1,2820737,Aarhus University; Linkopings Universitet,Aarhus University,Linkopings Universitet,None,None,None,None,None,None,None
2,2887054,Action contre La Faim,Action contre La Faim,None,None,None,None,None,None,None,None
3,4709373,Action contre La Faim,Action contre La Faim,None,None,None,None,None,None,None,None
4,3402155,ActionAid UK,ActionAid UK,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
968,4928397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
969,4961773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
970,4967556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
971,4991675,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
Projects.drop(['other_organisations'], axis=1, inplace=True)

#transforms the dataset from wide to long
Projects=pd.melt(Projects,id_vars=['project_id'], value_name='institution_name')

Projects = Projects.dropna()

Projects.drop(['variable'], axis=1, inplace=True)

#make all text in column lowercase
Projects["institution_name"] = Projects["institution_name"].str.lower()

#the w means any letter, the s means any whitespace and the ^ means 'not' so code below means
#remove all not letters and spaces from the column
Projects["institution_name"] = Projects["institution_name"].str.replace(r'[^a-z\s\d+]', '')

#remove words if less than 2 letters long
Projects['institution_name'] = Projects['institution_name'].apply(lambda x: ' '.join([word for word in x.split() if len(word)>1]))

#remove stopwords
Projects['institution_name'] = Projects['institution_name'].apply(lambda x: ' '.join([word for word in x.split() if not word in stopwords.words('english')]))

Projects

,project_id,institution_name
0,560204,3ie
1,2820737,aarhus university
2,2887054,action contre la faim
3,4709373,action contre la faim
4,3402155,actionaid uk
...,...,...
7139,2105816,spartans community football academy
7183,294704,council development social science research af...
7218,1443402,oneplusone
7303,2709176,university leeds


In [6]:
#remove unneeded words to increase matches
pattern = '|'.join(['university', 'universitet', 'college'])

Projects["institution_name"] = Projects["institution_name"].str.replace(pattern, '')
Universities["institution_name"] = Universities["institution_name"].str.replace(pattern, '')

Universities

,institution_name,country_code
0,tan kapuja buddhista fiskola tkbf,HU
1,aa school architecture,UK
2,aalborg,DK
3,aaltoyliopisto,FI
4,aarhus,DK
...,...,...
3643,,BG
3644,,BG
3645,,BG
3646,,BG


In [7]:
#This works for 508 of 832 records. 
def fuzzy_merge(baseFrame, compareFrame, baseKey, compareKey, threshold=90, limit=1, how='left'):
    #   baseFrame: the left table to join
    #   compareFrame: the right table to join
    #   baseKey: key column of the left table
    #   compareKey: key column of the right table
    #   threshold: how close the matches should be to return a match, based on Levenshtein distance
    #   limit: the amount of matches that will get returned, these are sorted high to low
    #   return: dataframe with boths keys and matches
    s_mapping = {x: fuzz_utils.default_process(x) for x in compareFrame[compareKey]}

    m1 = baseFrame[baseKey].apply(lambda x: process.extract(
      fuzz_utils.default_process(x), s_mapping, limit=limit, score_cutoff=threshold, processor=None
    ))
    baseFrame['Match'] = m1

    m2 = baseFrame['Match'].apply(lambda x: ', '.join(i[2] for i in x))
    baseFrame['organisations'] = m2.replace("",np.nan)

    return baseFrame.merge(compareFrame, left_on='organisations', right_on=baseKey, how=how)

Merged = fuzzy_merge(Projects, Universities, 'institution_name', 'institution_name')

Merged

,project_id,institution_name_x,Match,organisations,institution_name_y,country_code
0,560204,3ie,[],NaN,NaN,NaN
1,2820737,aarhus,"[(aarhus, 100.0, aarhus )]",aarhus,aarhus,DK
2,2887054,action contre la faim,[],NaN,NaN,NaN
3,4709373,action contre la faim,[],NaN,NaN,NaN
4,3402155,actionaid uk,[],NaN,NaN,NaN
...,...,...,...,...,...,...
827,2105816,spartans community football academy,[],NaN,NaN,NaN
828,294704,council development social science research af...,[],NaN,NaN,NaN
829,1443402,oneplusone,[],NaN,NaN,NaN
830,2709176,leeds,"[(leeds, 100.0, leeds)]",leeds,leeds,UK


In [8]:
Merged.to_csv('collaborations.csv')
#Collaborations.to_csv('university.csv')